<h1>Паутинообразная модель ценообразования с обучением<h1>
<h3>Работу выполнил студент 3-го курса ЭФ СПбГУ гр. 20.б03-э Шатров Д.И. в 2023 г.<h3>

Модель построена в стандартных предпосылках.<br>
Переменные состояния:<br>
* $P_t$ -- цена к началу года $t$;
* $S_t$ -- предложение к началу $t$;
* $D_t$ --спрос к началу $t$.<br>
Функционирование:
* $f_D(P)$ не возрастает;
* $f_S(P)$ не убывает;
* $\exists P^*:f_D(P^*)=f_S(P^*)$.<br>
Рассмотрим модель с обучением:
\begin{align*}
&D_t = A-BP_t+U_t\\
&S_t = C+E(P_{t-1}-k\Delta P_{t-1})+V_t\\
&\Delta P_{t-1}=P_{t-1}-P_{t-2}
\end{align*}

In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import sqrt
from IPython.display import display, Math
from scipy import linspace
import random
from collections import defaultdict

In [2]:
def suppress_print():
    sys.stdout = open(os.devnull, 'w')

# Restore
def restore_print():
    sys.stdout = sys.__stdout__

<h2>Функции спроса и предложения.</h2>
    Функции спроса и предложения создаются как экземпляры класса Dt для спроса и St для предложения. При создании функции нужно задать её неотрицательные параметры в терминах уравнений модели с обучениям. Для параметра $k$ обучения значение по умолчанию нулевое.<br><br>Ошибки $U_t,V_t$ являются гауссовским белым шумом. В текущем варианте модели иные распределения ошибок не предусмотрены. Дисперсия ошибки регулируется параметром sigma. По умолчанию значение параметра равно единице.<br><br>
    Вычисление значения функции осуществляется с помощью вызова метода $q$, которому на вход передаётся цена.
    <br><br>
    Метод $p$ класса Dt вычисляет цену покупки (продажи) товара при заданном объёме рынка.<br><br>
    Для обоих классов предусмотрен метод spec, возвращающий сразу весь список параметров (спецификацию) заданной функции спроса (предложения) и выводящий на экран запись уравнения в этой спецификации.

In [3]:
class Dt:
    def __init__(self, A, B, sigma = 1):
        assert(A>=0 and B>=0)
        self.A = A
        self.B = B
        self.sigma = sigma
    def q(self, P):
        assert P>=0, "Цена не может быть отрицательной!"
        U = np.random.normal(0, self.sigma)
        val = self.A - self.B * P + U
        if val>=0:
            return self.A - self.B * P + U
        else:
            return 0
    def p(self, Q):
        assert Q>=0, "Нельзя купить отрицательное количество товара!"
        U = np.random.normal(0,self.sigma)
        if self.B == 0:
            val = self.A + U
        else:
            val = (self.A + U - Q) / self.B
        if val>0:
            return val
        else:
            return 0
    def spec(self):
        #print("D_t = %0.2f-%0.2fP_t+U_t"%(self.A, self.B))
        display(Math(r"D_t={}-{}P_t+U_t".format(self.A, self.B)))
        return [self.A, self.B]
        

In [4]:
class St:
    def __init__(self, C, E, k = 0, sigma = 1):
        assert(C>=0 and E>=0 and k>=0)
        self.C = C
        self. E = E
        self.k = k
        self.sigma = sigma
    def q(self, P1, P2 = 0):
        assert P1>=0 and P2>=0, "Цена не может быть отрицательной!"
        V = np.random.normal(0, self.sigma)
        val =  self.C + self.E*P1-self.k * (P1-P2) + V
        if val>= 0:
            return val
        else:
            return 0
    def spec(self):
        if self.k == 0:
            #print("S_t = %0.2f+%0.2fP_(t-1)+V_t"%(self.C, self.E))
            display(Math(r"S_t={}+{}P_{{t-1}}+V_t".format(self.C, self.E)))
        else:
            #print("S_t = %0.2f-%0.2f[P_(t-1)-%0.2fdP_(t-1)]+V_t"%(self.C, self.E, self.k))
            display(Math(r"S_t={}+{}(P_{{t-1}}-{}\Delta P_{{t-1}})+V_t".format(self.C, self.E, self.k)))
        return [self.C, self.E, self.k]

In [5]:
#Функция спроса
spros = Dt(200,4,sigma = 1)
#Функция предложения
predl = St(40,3,0.05, sigma = 1)
spros.spec();
predl.spec();

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Функция cobweb принимает на вход начальную цену на рынке, функцию спроса и функцию предложения и возвращает список состояний рынка, выраженных объёмом рынка и ценой в каждый из периодов.

In [6]:
#собрать список состояний рынка, построить по точкам графики спроса и предложения, паутину
def cobweb(P, D, S, maxiter = 9000):
    flag = 0
    PQ = {"Цена":[],"Объём рынка":[S.q(P)]}
    iter = 0
    price = D.p(S.q(P))
    qty = S.q(price)
    PQ["Цена"].append(price)
    PQ["Объём рынка"].append(qty)
    price = D.p(qty)
    PQ["Цена"].append(price)
    while iter < maxiter:
        iter+=1
        qty = S.q(price)
        price = D.p(qty)
        if abs(qty - PQ["Объём рынка"][-1]) < min(PQ["Объём рынка"])/10000:
            print("Подбор остановлен!")
            break
        PQ["Цена"].append(price)
        PQ["Объём рынка"].append(qty)
    if abs(D.q(price) - qty) < min(PQ["Объём рынка"])/25 + max([sqrt(S.sigma), sqrt(D.sigma)]):
        print("Модель сошлась на значениях P = %0.2f, Q = %0.2f."%(PQ['Цена'][-1], PQ["Объём рынка"][-1]))
        flag = 1
    else:
        print("Модель не сошлась! Равновесие не найдено.")
        print("P:%0.2f, Q:%0.2f"%(PQ['Цена'][-1],PQ['Объём рынка'][-1]))
    print("Итераций:%d"%(iter))
    return PQ, flag

In [7]:
market, conv = cobweb(5,spros,predl)


Подбор остановлен!
Модель сошлась на значениях P = 22.85, Q = 107.19.
Итераций:91


Функция cws создаёт $m$ случайных паутинообразных моделей ценообразования с обучением, по $n$ раз проводит расчёты для каждой из них и сводит усреднённые результаты в таблицу. Таблица выводится на экран. Значения $m,n$ по умоланию равны десяти, но могут быть иными (в т.ч. различными).<br><br>
Дополнительно функции $cws$ могут быть переданы верхние границы для коэффициентов уравнений модели. Также можно передать точное значение дисперсии ошибок (она здесь фиксируется для всех моделей и функций).

In [8]:
def cws(m = 10, n = 10, Abound = 100, Bbound = 10, Cbound = 20, Ebound = 10, kbound = 0.05, s = 1):
    Aval, Bval, Cval, Eval, kval = ([] for i in range(5))
    searched = pd.DataFrame(columns = ["Спрос","Предложение","Сошлось (доля)", "Выпуск", "Цена"])
    for j in range(m):
        Aval.append(random.uniform(0, Abound))
        Bval.append(random.uniform(0, Bbound))
        Cval.append(random.uniform(0, Cbound))
        Eval.append(random.uniform(0, Ebound))
        kval.append(random.uniform(0, kbound))
        S_t = St(Cval[-1], Eval[-1], kval[-1], sigma = s)
        D_t = Dt(Aval[-1], Bval[-1], sigma = s)
        flagval, Qval, Pval = ([] for i in range(3))
        suppress_print()
        for t in range(n):
            PQ, flag = cobweb(3, D_t, S_t)
            flagval.append(flag)
            Qval.append(PQ["Объём рынка"][-1])
            Pval.append(PQ["Цена"][-1])
        modelres = {"Спрос":r"{:.2f}-{:.2f}P_t+U_t".format(D_t.A, D_t.B), 
                    "Предложение":r"{:.2f}+{:.2f}(P_{{t-1}}-{:.2f}dP_{{t-1}})+V_t".format(S_t.C, S_t.E, S_t.k),
                    "Сошлось (доля)":round(sum(flagval) / n,2),
                    "Выпуск":round(sum(Qval) / n,2),
                    "Цена":round(sum(Pval) / n,2)}
        #searched = searched.append(modelres, ignore_index=True)
        searched = pd.concat([searched, pd.DataFrame([modelres])], ignore_index=True)
        restore_print()
        
    return searched
    

In [9]:
cws(m = 10, n = 4)

,Спрос,Предложение,Сошлось (доля),Выпуск,Цена
0,4.50-6.10P_t+U_t,1.21+1.74(P_{t-1}-0.01dP_{t-1})+V_t,0.5,1.43,0.55
1,0.11-0.59P_t+U_t,0.75+7.88(P_{t-1}-0.02dP_{t-1})+V_t,0.0,3.96,0.0
2,99.19-1.13P_t+U_t,10.03+7.98(P_{t-1}-0.03dP_{t-1})+V_t,0.0,632.37,0.0
3,55.04-2.13P_t+U_t,3.22+1.82(P_{t-1}-0.05dP_{t-1})+V_t,1.0,26.73,13.28
4,49.21-5.42P_t+U_t,9.75+8.87(P_{t-1}-0.04dP_{t-1})+V_t,0.25,25.35,5.6
5,67.82-0.76P_t+U_t,16.20+6.90(P_{t-1}-0.03dP_{t-1})+V_t,0.0,491.22,0.0
6,72.57-5.75P_t+U_t,8.67+8.76(P_{t-1}-0.01dP_{t-1})+V_t,0.0,106.9,0.0
7,69.12-4.40P_t+U_t,1.52+4.33(P_{t-1}-0.00dP_{t-1})+V_t,0.5,37.66,7.1
8,88.16-5.71P_t+U_t,17.97+0.03(P_{t-1}-0.05dP_{t-1})+V_t,1.0,17.63,12.34
9,39.36-8.35P_t+U_t,19.44+3.67(P_{t-1}-0.02dP_{t-1})+V_t,0.5,25.61,1.68
